In [1]:
from PIL import Image, ImageOps
import pytesseract

In [2]:
import os
import cv2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import string

In [37]:
import re

In [179]:
import copy

In [428]:
import json

In [4]:
indir = './txt_boxes/test/'
for root, dirs, filenames in os.walk(indir):
    for filename in filenames:
        print('###################' + filename + '###################')
        im = Image.open(indir + filename)
        #region = crop_box(im)
        #text = pytesseract.image_to_string(region, lang='eng', config='--oem 1 --psm 1') #oem 1 means Neural Nets LSTM engine
        text = pytesseract.image_to_string(im, lang='eng')
        print(text)

###################frame46.jpg###################
KISS Mol Roe
W77? 02' 23.1"
###################frame47.jpg###################
KIS eto Roe
W77? 02' 23.1"
###################frame45.jpg###################
KIS Mtoe
W77? 02' 23.3"
###################frame44.jpg###################
LIS aa oX oy at
W77° 02' 24.2"
###################frame50.jpg###################
KISS etl Roe
W77° 02' 21.9"
###################frame40.jpg###################
IRIS aa X aos
77° 02' 30.9"
###################frame41.jpg###################
RIS ea eto Ro
W77? 02' 29.1"
###################frame43.jpg###################
N38° 53' 57.5"
W77° 02' 25.8"
###################frame42.jpg###################
DIS aatoX aol tb
W77? 02' 27.3"
###################frame25.jpg###################
N38° 54' 05.6"
77° 02' 49.9"
###################frame31.jpg###################
KISS Uo
W77° 02' 46.1"
###################frame30.jpg###################
DIS aak oe da Oo
W77° 02' 47.8"
###################frame24.jpg###################
KIS aaher

In [53]:
indir = './txt_boxes/test/'
for root, dirs, filenames in os.walk(indir):
    for filename in filenames:
        print('###################' + filename + '###################')
        image = cv2.imread(indir + filename)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        result = cv2.threshold(gray_image,120,255,cv2.THRESH_BINARY)[1]
        
        #new_image = cv2.threshold(gray_image,190,255,cv2.THRESH_BINARY)[1]
        #kernel = np.ones((1,1), np.uint8)
        #result = cv2.dilate(new_image, kernel, iterations=1)
        
        result = cv2.bitwise_not(result)
        
        #im = Image.open(indir + filename)
        #region = crop_box(im)
        #text = pytesseract.image_to_string(region, lang='eng', config='--oem 1 --psm 1') #oem 1 means Neural Nets LSTM engine
        
        #plt.imshow(result, cmap='Greys')
        #plt.show()
        
        text = pytesseract.image_to_string(result, lang='eng')
        print(text.split('\n')[0])

###################frame46.jpg###################
N38° 53' 56.5"
###################frame47.jpg###################
N38° 53' 56.5"
###################frame45.jpg###################
N38° 53' 56.8"
###################frame44.jpg###################
N38° §3' 57.1"
###################frame50.jpg###################
N38° 53' 55.5"
###################frame40.jpg###################
N38° 53' 59.1"
###################frame41.jpg###################
N38° §3' 58.6"
###################frame43.jpg###################
N38° 53' 57.5"
###################frame42.jpg###################
N38° 53' 58.1"
###################frame25.jpg###################
N38° 54' 05.6"
###################frame31.jpg###################
N38° 54' 04.6"
###################frame30.jpg###################
N38° 54' 05.1"
###################frame24.jpg###################
N38° 54' 06.0"
###################frame32.jpg###################
N38° 54' 03.9"
###################frame26.jpg###################
N38° 54' 05.4"
###################frame2

In [5]:
degree_sign = u'\N{DEGREE SIGN}'
print(degree_sign)

°


In [93]:
indir = './txt_boxes/gps_box/'

image_dict = {}
bad_lats = {}
bad_lons = {}
imgs_w_ie = []
degree_sign = u'\N{DEGREE SIGN}'
remove = string.punctuation + string.ascii_uppercase + degree_sign + '’' + '”'
remove = remove.replace(".", "") # don't remove periods

def lat_txt_to_float(line, bad_lats, dict_key):
    mod_string = line.translate({ord(char): None for char in remove})
    #print(mod_string)
    pattern = re.compile("(\d{2}\s\d{2}\s\d{2}\.\d)")
    if not(re.match(pattern, mod_string)):
        bad_lats[dict_key] = mod_string
    else:
        try:
            return float(mod_string.split(' ')[0]) + float(mod_string.split(' ')[1])/60 + \
                     float(mod_string.split(' ')[2][:4])/3600
        except ValueError as e:
            raise ValueError(f"Check image: {filename.split('/')[-1]}") from e
            
def lon_txt_to_float(line, bad_lons, dict_key):
    mod_string = line.translate({ord(char): None for char in remove})
    #print(mod_string)
    pattern = re.compile("(\d{2,3}\s\d{2}\s\d{2}\.\d)")
    if not(re.match(pattern, mod_string)):
        bad_lons[dict_key] = mod_string
    else:
        try:
            return float(mod_string.split(' ')[0]) + float(mod_string.split(' ')[1])/60 + \
                     float(mod_string.split(' ')[2][:4])/3600
        except ValueError as e:
            raise ValueError(f"Check image: {filename.split('/')[-1]}") from e

loop_counter = 0

for entry in os.scandir(indir):
    loop_counter += 1
    if loop_counter % 100 == 0:
        print("Loop counter: ", loop_counter)
    GPS_dict = {}
    filename = entry.path
    #print('###################' + filename.split('/')[-1] + '###################')
    image = cv2.imread(filename)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    result = cv2.threshold(gray_image,120,255,cv2.THRESH_BINARY)[1]

    #new_image = cv2.threshold(gray_image,190,255,cv2.THRESH_BINARY)[1]
    #kernel = np.ones((1,1), np.uint8)
    #result = cv2.dilate(new_image, kernel, iterations=1)

    result = cv2.bitwise_not(result)

    #im = Image.open(indir + filename)
    #region = crop_box(im)
    #text = pytesseract.image_to_string(region, lang='eng', config='--oem 1 --psm 1') #oem 1 means Neural Nets LSTM engine

    #plt.imshow(result, cmap='Greys')
    #plt.show()
                             
    #text = pytesseract.image_to_string(result, lang='eng', config='-c tessedit_char_whitelist=’WN0123456789.° --oem 0')
    text = pytesseract.image_to_string(result, lang='eng', config="""-c tessedit_char_whitelist=’WN0123456789.°\\'\\" --oem 0""")
    #text = pytesseract.image_to_string(result, lang='eng')
    
    dict_key = filename.split('/')[-1].split('.')[0]
    
    try:
        GPS_dict['Lat'] = lat_txt_to_float(text.split('\n')[0], bad_lats, dict_key)
    except IndexError:
        imgs_w_ie.append({dict_key:('Lat',text)})

    try:
        GPS_dict['Lon'] = lon_txt_to_float(text.split('\n')[1], bad_lons, dict_key)
    except IndexError:
        imgs_w_ie.append({dict_key:('Lon',text)})
    
    image_dict[dict_key] = GPS_dict
    
    #print(text.split('\n')[0])
    
#print(bad_lats)
#print(bad_lons)

Loop counter:  100
Loop counter:  200
Loop counter:  300
Loop counter:  400
Loop counter:  500
Loop counter:  600
Loop counter:  700
Loop counter:  800
Loop counter:  900
Loop counter:  1000
Loop counter:  1100
Loop counter:  1200
Loop counter:  1300
Loop counter:  1400
Loop counter:  1500
Loop counter:  1600
Loop counter:  1700
Loop counter:  1800
Loop counter:  1900
Loop counter:  2000
Loop counter:  2100
Loop counter:  2200
Loop counter:  2300
Loop counter:  2400
Loop counter:  2500
Loop counter:  2600
Loop counter:  2700
Loop counter:  2800
Loop counter:  2900
Loop counter:  3000
Loop counter:  3100
Loop counter:  3200
Loop counter:  3300
Loop counter:  3400
Loop counter:  3500
Loop counter:  3600
Loop counter:  3700
Loop counter:  3800
Loop counter:  3900
Loop counter:  4000
Loop counter:  4100
Loop counter:  4200
Loop counter:  4300
Loop counter:  4400
Loop counter:  4500
Loop counter:  4600
Loop counter:  4700
Loop counter:  4800
Loop counter:  4900
Loop counter:  5000
Loop coun

In [100]:
len(image_dict)

39524

In [94]:
len(bad_lons)

5089

In [139]:
#print a 10-sample of key-value pairs from bad_lons
{k: v for i, (k,v) in enumerate(dict((k, bad_lons[k]) for k in bad_lons.keys()).items()) if i < 10}

{'frame28907': '111 42 21 .6',
 'frame6817': '84 00 35. 2 3',
 'frame14998': '94 35 51 .9',
 'frame17249': '97 52 41 .3',
 'frame19516': '1011128.8.',
 'frame16625': '96 59 241',
 'frame6195': '831052.5 .',
 'frame39188': '122 09 01 .8',
 'frame18146': '99 18 41 .6',
 'frame13013': '91 41 58 4'}

In [140]:
#print a 10-sample of key-value pairs from bad_lats
{k: v for i, (k,v) in enumerate(dict((k, bad_lats[k]) for k in bad_lats.keys()).items()) if i < 10}

{'frame30190': '41  50 43.0',
 'frame8184': '3815 00.1 5',
 'frame6817': '38 04 03. 7  .',
 'frame10532': '38 19 01 .0',
 'frame25589': '4010 21.1',
 'frame13985': '38 57 31 .0',
 'frame30184': '41  50 18.6',
 'frame3953': '37 43 51 .8',
 'frame13013': '38 54 41 .5',
 'frame24867': '39 4113.5 '}

In [95]:
len(bad_lats)

2075

In [101]:
#how many frames have both poorly OCR'd latitude and longitude values
len(bad_lats.keys() & bad_lons.keys())

486

In [103]:
len(image_dict.keys() & bad_lats.keys())

2075

In [104]:
len(image_dict.keys() & bad_lons.keys())

5089

In [121]:
#size of dictionary containing key-value pairs from image_dict for keys that are also found in bad_lats dictionary
len(dict((k, image_dict[k]) for k in (image_dict.keys() & bad_lats.keys())))

2075

In [117]:
#print a 10-sample of key-value pairs from image_dict for keys that are also found in bad_lats dictionary
{k: v for i, (k,v) in enumerate(dict((k, image_dict[k]) for k in (image_dict.keys() & bad_lats.keys())).items()) if i < 10}

{'frame8359': {'Lat': None, 'Lon': 85.89155555555556},
 'frame6845': {'Lat': None, 'Lon': 84.04952777777778},
 'frame7862': {'Lat': None, 'Lon': 85.25738888888888},
 'frame32668': {'Lat': None, 'Lon': 115.90894444444444},
 'frame467': {'Lat': None, 'Lon': 77.46622222222223},
 'frame36963': {'Lat': None, 'Lon': None},
 'frame24656': {'Lat': None, 'Lon': 107.78122222222223},
 'frame6392': {'Lat': None, 'Lon': 83.43738888888889},
 'frame9776': {'Lat': None, 'Lon': 87.869},
 'frame5612': {'Lat': None, 'Lon': 82.36247222222222}}

In [136]:
n_none_lats, n_none_lons, n_none_both = (0,0,0)
#subset image_dict to frames that are NOT found in imgs_w_ie list of dictionaries
missing_keys = set().union(*(d.keys() for d in imgs_w_ie))
image_dict_rstr = dict((k, image_dict[k]) for k in (image_dict.keys() - missing_keys))
#count frames in image_dict that have None values in one or both of Lat and Lon
for gps_dict in image_dict_rstr.values():
    if (gps_dict['Lat'] is None) & (gps_dict['Lon'] is None):
        n_none_both += 1
    elif gps_dict['Lat'] is None:
        n_none_lats += 1
    elif gps_dict['Lon'] is None:
        n_none_lons += 1
print("Only Lat none:", n_none_lats, "\nOnly Lon none:", n_none_lons, "\nBoth none:", n_none_both)

Only Lat none: 1571 
Only Lon none: 4603 
Both none: 486


In [137]:
len(image_dict_rstr)

39506

In [138]:
#subset bad_lats dictionary to frames that are NOT found in imgs_w_ie list of dictionaries
missing_keys = set().union(*(d.keys() for d in imgs_w_ie))
bad_lats_rstr = dict((k, image_dict[k]) for k in (bad_lats.keys() - missing_keys))
len(bad_lats_rstr)

2057

In [96]:
lat_n_digits = {}
for value in bad_lats.values():
    n_digits = sum(c.isdigit() for c in value)
    if n_digits not in lat_n_digits:
        lat_n_digits[n_digits] = 1
    else:
        lat_n_digits[n_digits] += 1
print(lat_n_digits)

{7: 1757, 8: 274, 9: 11, 5: 3, 1: 6, 0: 20, 6: 3, 4: 1}


In [87]:
for key, value in bad_lats.items():
    if sum(c.isdigit() for c in value) not in (7,8):
        print(key, ',', value)

frame12124 , 38 381371 1
frame147 , 1
frame37987 , 47 0145.41 5
frame4623 , 38 130211 1
frame8151 , 381419.5 5 1
frame2108 , 38 03 2411 1
frame9470 , 38 10 2570 3
frame8961 , 38 124941 1
frame22008 , 
frame5473 , 38 25 1
frame8619 , 38 143013 1
frame39538 , 
frame39539 , 
frame39103 , 47 331177 5
frame22735 , 1
frame5589 , 38 251911 1
frame3271 , 37 47 15. 
frame652 , 
frame39529 , 
frame39531 , 
frame5541 , 38 24 58 
frame39530 , 
frame12010 , 
frame39532 , 
frame29292 , 7
frame39533 , 
frame9755 , 38 11 33 01 1
frame39537 , 
frame39536 , 
frame8128 , 38114010.0
frame39534 , 
frame39051 , 47 32 173.4 3
frame39535 , 
frame4288 , 1
frame4370 , 38 05 
frame24884 , 39 42 010.0 1 
frame4373 ,  05 58.8
frame39021 , 47 32 419. 5 5
frame4428 , 38 06 51 .0 11
frame3347 , 4
frame39544 , 
frame4362 , 38 051773  1
frame4438 , 3807 28.7 51
frame38138 , 47 034 22. 54 5
frame39540 , 
frame23914 , 39 39 010.0 5
frame39541 , 
frame5484 , 38 25 00.  .
frame39543 , 
frame39542 , 
frame23297 , 39 33 4 1


In [97]:
lon_n_digits = {}
for value in bad_lons.values():
    n_digits = sum(c.isdigit() for c in value)
    if n_digits not in lon_n_digits:
        lon_n_digits[n_digits] = 1
    else:
        lon_n_digits[n_digits] += 1
print(lon_n_digits)

{8: 2769, 7: 1912, 10: 16, 5: 8, 9: 154, 0: 157, 6: 72, 11: 1}


In [98]:
lon_8_digits = {'8, starts with 1': 0, '8, starts with other': 0}
for value in bad_lons.values():
    n_digits = sum(c.isdigit() for c in value)
    if n_digits == 8:
        first_digit_1 = (value[0] == '1')
        if first_digit_1:
            lon_8_digits['8, starts with 1'] += 1
        else:
            lon_8_digits['8, starts with other'] += 1
print(lon_8_digits)

{'8, starts with 1': 2454, '8, starts with other': 315}


In [92]:
for key, value in bad_lons.items():
    if sum(c.isdigit() for c in value) == 9:
        print(key, ',', value)

frame36536 , 1191431.81
frame22543 , 105 25 3515
frame38916 , 121 491832
frame36873 , 119 44 2313
frame22351 , 1051118.61
frame36324 , 1191815.31
frame36325 , 1191810.05
frame7671 , 85 001774 1
frame19661 , 101 25.12.33
frame38877 , 121  46 27.6.1
frame19932 , 101 5213.33
frame20380 , 102 3410.03
frame19663 , 101  25 24.31
frame38861 , 121 44. 47.94
frame39202 , 122 09. 50.41
frame38450 , 12116 23.77
frame38322 , 121 05 010.0.
frame27564 , 110 23. 54.41
frame25400 , 108 12 010.0
frame24087 , 107 11.21.87
frame37560 , 120 28 3251
frame39217 , 122 100311
frame19662 , 101 2918.31
frame28246 , 11115 24.65
frame19058 , 100 41 08 31
frame38531 , 121 21  13.01
frame24124 , 107 13. 25.71
frame19276 , 100 54 010.0
frame20420 , 102 33. 40.21
frame28256 , 11115 39.21
frame19666 , 101 25. 42.33
frame36336 , 1191710.21
frame39010 , 121 57. 03.51
frame22418 , 105 1558.41
frame35414 , 118 27. 52.03
frame27819 , 110 45. 02.71
frame18828 , 100 1335.01
frame28080 , 111 0917.82
frame31512 , 114 1628.31
f

In [99]:
len(imgs_w_ie)

18

In [76]:
imgs_w_ie

[{'frame22008': ('Lon', '')},
 {'frame39538': ('Lon', '')},
 {'frame39539': ('Lon', '')},
 {'frame39529': ('Lon', '')},
 {'frame39531': ('Lon', '')},
 {'frame39530': ('Lon', '')},
 {'frame12010': ('Lon', '')},
 {'frame39532': ('Lon', '')},
 {'frame39533': ('Lon', '')},
 {'frame39537': ('Lon', '')},
 {'frame39536': ('Lon', '')},
 {'frame39534': ('Lon', '')},
 {'frame39535': ('Lon', '')},
 {'frame39544': ('Lon', '')},
 {'frame39540': ('Lon', '')},
 {'frame39541': ('Lon', '')},
 {'frame39543': ('Lon', '')},
 {'frame39542': ('Lon', '')}]

In [127]:
#extract keys of all dictionaries present in imgs_w_ie list of dictionaries
all_keys = set().union(*(d.keys() for d in imgs_w_ie))

In [128]:
all_keys

{'frame12010',
 'frame22008',
 'frame39529',
 'frame39530',
 'frame39531',
 'frame39532',
 'frame39533',
 'frame39534',
 'frame39535',
 'frame39536',
 'frame39537',
 'frame39538',
 'frame39539',
 'frame39540',
 'frame39541',
 'frame39542',
 'frame39543',
 'frame39544'}

In [38]:
pattern = re.compile("(\d{2,3}\s\d{2}\s\d{2}\.\d)")

In [46]:
if re.match(pattern, '77 14 08.2'):
    print("yes")
else:
    print("no")

yes


In [51]:
if not(re.match(pattern, '77 14 08 2')):
    print("no")

no


In [48]:
#dpi = im.info

#indir = './frames_lmtd/'
#im_full = Image.open(indir + 'frame40.jpg')

#dpi = im_full.info['dpi']

In [106]:
n_stamps_per_frame = {}
for key, value in image_dict.items():
    if len(value) not in n_stamps_per_frame:
        n_stamps_per_frame[len(value)] = 1
    else:
        n_stamps_per_frame[len(value)] += 1
n_stamps_per_frame

{2: 39506, 1: 18}

### Fix OCR'd text in latitude and longitude stamps

In [141]:
#print a 10-sample of key-value pairs from bad_lats
{k: v for i, (k,v) in enumerate(dict((k, bad_lats[k]) for k in bad_lats.keys()).items()) if i < 10}

{'frame30190': '41  50 43.0',
 'frame8184': '3815 00.1 5',
 'frame6817': '38 04 03. 7  .',
 'frame10532': '38 19 01 .0',
 'frame25589': '4010 21.1',
 'frame13985': '38 57 31 .0',
 'frame30184': '41  50 18.6',
 'frame3953': '37 43 51 .8',
 'frame13013': '38 54 41 .5',
 'frame24867': '39 4113.5 '}

In [142]:
#make a shallow copy of bad_lats dictionary
bad_lats_copy = bad_lats.copy()

In [144]:
#remove all frames from bad_lats where number of digits in latitude value is not 7
bad_lats_copy = {k:v for (k,v) in bad_lats_copy.items() if sum(c.isdigit() for c in v) == 7}

In [145]:
len(bad_lats_copy)

1757

In [148]:
#remove all spaces from values
bad_lats_copy = {k: v.replace(" ","") for (k,v) in bad_lats_copy.items()}

In [149]:
#check all existing positions of a period in values
pos_of_period = {}
for v in bad_lats_copy.values():
    pos_list = [pos for pos, char in enumerate(v) if char == '.']
    if tuple(pos_list) not in pos_of_period:
        pos_of_period[tuple(pos_list)] = 1
    else:
        pos_of_period[tuple(pos_list)] += 1
pos_of_period

{(6,): 1602, (6, 8): 17, (): 133, (5, 7): 2, (7,): 2, (5,): 1}

In [150]:
sum(pos_of_period.values())

1757

In [151]:
#check latitude stamps that don't have periods
for v in bad_lats_copy.values():
    if '.' not in v:
        print(v)

3833354
3754502
3810148
3829074
3812222
4607586
3819132
4640594
3932074
3811585
3826415
3853404
3804134
3821094
4718152
4640514
3851291
3906426
3852413
3944325
3853064
4659133
4605395
4548054
3823191
3824175
3932454
3829192
4731533
3807092
4019172
4735002
4632213
4643214
3940354
4711119
3826096
3819502
3819382
3807404
4725393
3803452
3945171
4012010
3748502
4723313
3814262
3759211
3826594
4503115
3931514
4554165
3937403
4543263
4318134
3854412
4551452
4733323
3338355
4716132
3816432
4016584
3800164
4714050
3748203
4635134
3933332
4733263
3802155
3931212
4733024
3813232
4603464
4322392
3931431
3839073
4724598
4552102
3749324
3819442
4648072
4715539
4024122
4724565
4733473
3801114
3800534
4600583
4715151
4638291
3816382
3748526
4549575
3941444
4729431
4019563
4642318
4639493
4606482
4520522
4734525
3836204
3823214
3553133
3825171
3819274
3933452
3822133
4728248
3810114
3944433
3824583
4734481
4536413
3854122
3815594
3804290
3825443
3755285
4640173
3852111
3838244
4734452
4734415
3836262


In [152]:
#print a 10-sample of key-value pairs from bad_lats_copy
{k: v for i, (k,v) in enumerate(dict((k, bad_lats_copy[k]) for k in bad_lats_copy.keys()).items()) if i < 10}

{'frame30190': '415043.0',
 'frame6817': '380403.7.',
 'frame10532': '381901.0',
 'frame25589': '401021.1',
 'frame13985': '385731.0',
 'frame30184': '415018.6',
 'frame3953': '374351.8',
 'frame13013': '385441.5',
 'frame24867': '394113.5',
 'frame17261': '385241.9'}

In [155]:
#remove all frames where position of period is anything other than (6,) or (6,8), or there is no period at all
bad_lats_copy = {k: v for (k,v) in bad_lats_copy.items() if tuple([pos for pos, char in enumerate(v) if char == '.'])
    in ((6,),(6,8),())}

In [157]:
len(bad_lats_copy)

1752

In [159]:
#function to remove trailing periods, to insert period for stamps that need it, to insert spaces,
#and to convert text to float
def lat_clean_transform(stamp):
    no_trail_prd = stamp[:8]
    if '.' not in no_trail_prd:
        stamp_new = no_trail_prd[:2] + ' ' + no_trail_prd[2:4] + ' ' + no_trail_prd[4:6] + '.' + no_trail_prd[6:6]
    else:
        stamp_new = no_trail_prd[:2] + ' ' + no_trail_prd[2:4] + ' ' + no_trail_prd[4:8]
    return float(stamp_new.split(' ')[0]) + float(stamp_new.split(' ')[1])/60 + \
                     float(stamp_new.split(' ')[2])/3600

In [346]:
replaced_lats = {k: lat_clean_transform(v) for (k,v) in bad_lats_copy.items()}

In [347]:
len(replaced_lats)

1752

In [348]:
#print a 10-sample of key-value pairs from replaced_lats
{k: v for i, (k,v) in enumerate(dict((k, replaced_lats[k]) for k in replaced_lats.keys()).items()) if i < 10}

{'frame30190': 41.84527777777778,
 'frame6817': 38.06769444444445,
 'frame10532': 38.316944444444445,
 'frame25589': 40.17252777777777,
 'frame13985': 38.95861111111111,
 'frame30184': 41.8385,
 'frame3953': 37.73105555555556,
 'frame13013': 38.91152777777778,
 'frame24867': 39.68708333333333,
 'frame17261': 38.878305555555556}

In [349]:
sum(x is None for x in replaced_lats.values())

0

In [350]:
#print a 10-sample of key-value pairs from bad_lons
{k: v for i, (k,v) in enumerate(dict((k, bad_lons[k]) for k in bad_lons.keys()).items()) if i < 10}

{'frame28907': '111 42 21 .6',
 'frame6817': '84 00 35. 2 3',
 'frame14998': '94 35 51 .9',
 'frame17249': '97 52 41 .3',
 'frame19516': '1011128.8.',
 'frame16625': '96 59 241',
 'frame6195': '831052.5 .',
 'frame39188': '122 09 01 .8',
 'frame18146': '99 18 41 .6',
 'frame13013': '91 41 58 4'}

In [351]:
#make a shallow copy of bad_lons dictionary
bad_lons_copy = bad_lons.copy()

In [352]:
#remove all frames from bad_lons where number of digits in longitude value is not 7 or 8 with first digit being 1
bad_lons_copy = {k:v for (k,v) in bad_lons_copy.items() if (sum(c.isdigit() for c in v) == 7) |
                ((sum(c.isdigit() for c in v) == 8) & (v.startswith('1')))}

In [353]:
len(bad_lons_copy)

4366

In [354]:
#remove all spaces from values
bad_lons_copy = {k: v.replace(" ","") for (k,v) in bad_lons_copy.items()}

In [355]:
#print a 10-sample of key-value pairs from bad_lons_copy
{k: v for i, (k,v) in enumerate(dict((k, bad_lons_copy[k]) for k in bad_lons_copy.keys()).items()) if i < 10}

{'frame28907': '1114221.6',
 'frame14998': '943551.9',
 'frame17249': '975241.3',
 'frame19516': '1011128.8.',
 'frame16625': '9659241',
 'frame6195': '831052.5.',
 'frame39188': '1220901.8',
 'frame18146': '991841.6',
 'frame13013': '9141584',
 'frame15476': '951731.7'}

In [356]:
#check all existing positions of a period in values
pos_of_period = {}
for v in bad_lons_copy.values():
    pos_list = [pos for pos, char in enumerate(v) if char == '.']
    if tuple(pos_list) not in pos_of_period:
        pos_of_period[tuple(pos_list)] = 1
    else:
        pos_of_period[tuple(pos_list)] += 1
pos_of_period

{(7,): 2344,
 (6,): 1434,
 (7, 9): 37,
 (): 455,
 (6, 8): 75,
 (8,): 14,
 (5,): 1,
 (6, 8, 9): 5,
 (6, 7): 1}

In [357]:
sum(pos_of_period.values())

4366

In [358]:
#check longitude stamps that don't have periods
for v in bad_lons_copy.values():
    if '.' not in v:
        print(v)

9659241
9141584
8004543
8343403
9055105
10759095
11010461
9040162
8555335
9109330
9933271
8502563
8128182
11944233
12129451
9030196
11917431
8202494
8214553
12157381
8103243
7947112
12209427
10046004
10623451
9836023
1075912
8150303
8058364
8449523
8136424
9314294
11106112
8112522
7844553
9056382
8112509
7822405
9644483
8503105
9937173
1062435
7707225
7905002
9057341
8115520
7859345
8017153
12117341
10251301
10709225
7854257
12113411
7903336
8032210
9537118
8823395
9937221
1092147
8422093
7750571
9933388
7754191
9638234
9040071
11106112
8311021
11104142
7920410
9433348
8113138
8205415
8540341
8309232
9333547
9039561
1175735
9549534
8328072
7756285
8313561
8229332
8115213
1093702
9758256
11024225
11912570
11913153
7904310
1045901
11026221
9937395
8118423
7954403
8803306
8204372
8059231
8113105
11104192
8147241
9716533
8058512
9331271
9039592
9858110
11115397
8952335
9343535
8514136
9440334
9540179
7705449
9234421
7852503
10701183
8500335
12210013
7946455
8313471
8514183
8421591
8205335


In [359]:
#remove all frames where position of period is confusing
bad_lons_copy = {k: v for (k,v) in bad_lons_copy.items() if tuple([pos for pos, char in enumerate(v) if char == '.'])
    in ((7,),(6,),(7,9),(6,8),())}

In [360]:
len(bad_lons_copy)

4345

In [361]:
#print a 10-sample of key-value pairs from bad_lons_copy
{k: v for i, (k,v) in enumerate(dict((k, bad_lons_copy[k]) for k in bad_lons_copy.keys()).items()) if i < 10}

{'frame28907': '1114221.6',
 'frame14998': '943551.9',
 'frame17249': '975241.3',
 'frame19516': '1011128.8.',
 'frame16625': '9659241',
 'frame6195': '831052.5.',
 'frame39188': '1220901.8',
 'frame18146': '991841.6',
 'frame13013': '9141584',
 'frame15476': '951731.7'}

In [321]:
#len({k: v for (k,v) in bad_lons_copy.items() if len(v) < 7})

In [362]:
#function to remove trailing periods, to insert period for stamps that need it, to insert spaces,
#and to convert text to float
def lon_clean_transform(stamp):
    if (sum(c.isdigit() for c in stamp) == 7) & (tuple([pos for pos, char in enumerate(stamp) if char == '.']) in ((6,),(6,8),())):
        no_trail_prd = stamp[:8]
        if '.' not in no_trail_prd:
            stamp_new = no_trail_prd[:2] + ' ' + no_trail_prd[2:4] + ' ' + no_trail_prd[4:6] + '.' + no_trail_prd[6:6]
        else:
            stamp_new = no_trail_prd[:2] + ' ' + no_trail_prd[2:4] + ' ' + no_trail_prd[4:8]
        return float(stamp_new.split(' ')[0]) + float(stamp_new.split(' ')[1])/60 + \
                         float(stamp_new.split(' ')[2])/3600
    elif (sum(c.isdigit() for c in stamp) == 8) & (tuple([pos for pos, char in enumerate(stamp) if char == '.']) in ((7,),(7,9),())):
        no_trail_prd = stamp[:9]
        if '.' not in no_trail_prd:
            stamp_new = no_trail_prd[:3] + ' ' + no_trail_prd[3:5] + ' ' + no_trail_prd[5:7] + '.' + no_trail_prd[7:7]
        else:
            stamp_new = no_trail_prd[:3] + ' ' + no_trail_prd[3:5] + ' ' + no_trail_prd[5:9]
        return float(stamp_new.split(' ')[0]) + float(stamp_new.split(' ')[1])/60 + \
                         float(stamp_new.split(' ')[2])/3600

In [363]:
replaced_lons = {k: lon_clean_transform(v) for (k,v) in bad_lons_copy.items()}

In [364]:
len(replaced_lons)

4345

In [365]:
#print a 10-sample of key-value pairs from replaced_lons
{k: v for i, (k,v) in enumerate(dict((k, replaced_lons[k]) for k in replaced_lons.keys()).items()) if i < 10}

{'frame28907': 111.706,
 'frame14998': 94.59774999999999,
 'frame17249': 97.87813888888888,
 'frame19516': 101.19133333333333,
 'frame16625': 96.99,
 'frame6195': 83.18125,
 'frame39188': 122.15050000000001,
 'frame18146': 99.31155555555556,
 'frame13013': 91.69944444444445,
 'frame15476': 95.29213888888889}

In [366]:
sum(x is None for x in replaced_lons.values())

22

In [367]:
#bad_lons_copy['frame28907']

In [368]:
replaced_lats = {k: {'Lat':v} for (k,v) in replaced_lats.items()}

In [369]:
replaced_lons = {k: {'Lon':v} for (k,v) in replaced_lons.items()}

In [370]:
#print a 10-sample of key-value pairs from replaced_lats
{k: v for i, (k,v) in enumerate(dict((k, replaced_lats[k]) for k in replaced_lats.keys()).items()) if i < 10}

{'frame30190': {'Lat': 41.84527777777778},
 'frame6817': {'Lat': 38.06769444444445},
 'frame10532': {'Lat': 38.316944444444445},
 'frame25589': {'Lat': 40.17252777777777},
 'frame13985': {'Lat': 38.95861111111111},
 'frame30184': {'Lat': 41.8385},
 'frame3953': {'Lat': 37.73105555555556},
 'frame13013': {'Lat': 38.91152777777778},
 'frame24867': {'Lat': 39.68708333333333},
 'frame17261': {'Lat': 38.878305555555556}}

In [371]:
#print a 10-sample of key-value pairs from replaced_lons
{k: v for i, (k,v) in enumerate(dict((k, replaced_lons[k]) for k in replaced_lons.keys()).items()) if i < 10}

{'frame28907': {'Lon': 111.706},
 'frame14998': {'Lon': 94.59774999999999},
 'frame17249': {'Lon': 97.87813888888888},
 'frame19516': {'Lon': 101.19133333333333},
 'frame16625': {'Lon': 96.99},
 'frame6195': {'Lon': 83.18125},
 'frame39188': {'Lon': 122.15050000000001},
 'frame18146': {'Lon': 99.31155555555556},
 'frame13013': {'Lon': 91.69944444444445},
 'frame15476': {'Lon': 95.29213888888889}}

In [372]:
sum(x == {'Lat': None} for x in replaced_lats.values())

0

In [373]:
sum(x == {'Lon': None} for x in replaced_lons.values())

22

In [374]:
len(replaced_lats.keys() & replaced_lons.keys())

256

In [375]:
len(replaced_lats.keys() - replaced_lons.keys())

1496

In [376]:
len(replaced_lons.keys() - replaced_lats.keys())

4089

In [413]:
image_dict_final = copy.deepcopy(image_dict_rstr)

In [414]:
#number of keys in value dictionary in image_dict_final
keys_0 = 0
keys_1 = 0
keys_2 = 0
for v in image_dict_final.values():
    n_keys = len(v.keys())
    if n_keys == 0:
        keys_0 += 1
    elif n_keys == 1:
        keys_1 += 1
    elif n_keys == 2:
        keys_2 += 2
print("0 keys:",keys_0,"\n1 key:",keys_1,"\n2 keys:",keys_2)

0 keys: 0 
1 key: 0 
2 keys: 79012


In [415]:
#number of keys in value dictionary in replaced_lons
keys_0 = 0
keys_1 = 0
keys_2 = 0
for v in replaced_lons.values():
    n_keys = len(v.keys())
    if n_keys == 0:
        keys_0 += 1
    elif n_keys == 1:
        keys_1 += 1
    elif n_keys == 2:
        keys_2 += 2
print("0 keys:",keys_0,"\n1 key:",keys_1,"\n2 keys:",keys_2)

0 keys: 0 
1 key: 4345 
2 keys: 0


In [416]:
#print a 10-sample of key-value pairs from image_dict_final
{k: v for i, (k,v) in enumerate(dict((k, image_dict_final[k]) for k in image_dict_final.keys()).items()) if i < 10}

{'frame24135': {'Lat': 39.56769444444445, 'Lon': 107.23069444444445},
 'frame21410': {'Lat': 39.54383333333333, 'Lon': 103.99863888888889},
 'frame13131': {'Lat': 38.94269444444444, 'Lon': 91.87630555555555},
 'frame12203': {'Lat': 38.67222222222222, 'Lon': 90.66386111111112},
 'frame2400': {'Lat': 37.801111111111105, 'Lon': None},
 'frame26805': {'Lat': 40.41894444444444, 'Lon': 109.58152777777778},
 'frame24656': {'Lat': None, 'Lon': 107.78122222222223},
 'frame34179': {'Lat': 44.51555555555556, 'Lon': 117.37480555555555},
 'frame37262': {'Lat': 46.41525, 'Lon': 120.30847222222222},
 'frame3809': {'Lat': 37.76766666666666, 'Lon': None}}

In [417]:
#check how many frames have no latitude or longitude value in image_dict_final before fixed values are inserted
lon_none = 0
lat_none = 0
lat_lon_none = 0
for v in image_dict_final.values():
    if (v['Lat'] is None) & (v['Lon'] is None):
        lat_lon_none += 1
    elif v['Lat'] is None:
        lat_none += 1
    elif v['Lon'] is None:
        lon_none += 1
print("Frames without Lat:",lat_none,"\nFrames without Lon:",lon_none,"\nFrames without both:",lat_lon_none)

Frames without Lat: 1571 
Frames without Lon: 4603 
Frames without both: 486


In [418]:
#copy fixed latitude and longitude stamps into final image_dict_final dictionary
for k, v in image_dict_final.items():
    if (v['Lat'] is None) & (k in replaced_lats):
        image_dict_final[k]['Lat'] = replaced_lats[k]['Lat']
    if (v['Lon'] is None) & (k in replaced_lons):
        image_dict_final[k]['Lon'] = replaced_lons[k]['Lon']

In [419]:
#check how many frames ended up having no latitude or longitude value
lon_none = 0
lat_none = 0
lat_lon_none = 0
for v in image_dict_final.values():
    if (v['Lat'] is None) & (v['Lon'] is None):
        lat_lon_none += 1
    elif v['Lat'] is None:
        lat_none += 1
    elif v['Lon'] is None:
        lon_none += 1
print("Frames without Lat:",lat_none,"\nFrames without Lon:",lon_none,"\nFrames without both:",lat_lon_none)

Frames without Lat: 234 
Frames without Lon: 695 
Frames without both: 71


In [420]:
len(image_dict_final)

39506

In [421]:
#remove frames without latitude and/or longitude value
for k in list(image_dict_final.keys()):
    if (image_dict_final[k]['Lat'] is None) | (image_dict_final[k]['Lon'] is None):
        del image_dict_final[k]

In [422]:
len(image_dict_final)

38506

In [423]:
#check how many remaining latitude and longitude values fall outside the expected bounds
lat_out_of_bounds = 0
lon_out_of_bounds = 0
both_out_of_bounds = 0
for v in image_dict_final.values():
    if ((v['Lat'] < 37.0) | (v['Lat'] > 47.7)) & ((v['Lon'] < 77.0) | (v['Lon'] > 122.33)):
        both_out_of_bounds += 1
    elif (v['Lat'] < 37.0) | (v['Lat'] > 47.7):
        lat_out_of_bounds += 1
    elif (v['Lon'] < 77.0) | (v['Lon'] > 122.33):
        lon_out_of_bounds += 1
print("Both of out bounds:",both_out_of_bounds,"\nLat out of bounds:",lat_out_of_bounds,"\nLon out of bounds:",lon_out_of_bounds)

Both of out bounds: 1 
Lat out of bounds: 28 
Lon out of bounds: 34


In [424]:
#remove these frames that have latitude and longitude values fall outside the expected bounds
for k in list(image_dict_final.keys()):
    if ((image_dict_final[k]['Lat'] < 37.0) | (image_dict_final[k]['Lat'] > 47.7) | 
        (image_dict_final[k]['Lon'] < 77.0) | (image_dict_final[k]['Lon'] > 122.33)):
        del image_dict_final[k]

In [425]:
len(image_dict_final)

38443

In [426]:
#print a 10-sample of key-value pairs from image_dict_final
{k: v for i, (k,v) in enumerate(dict((k, image_dict_final[k]) for k in image_dict_final.keys()).items()) if i < 10}

{'frame24135': {'Lat': 39.56769444444445, 'Lon': 107.23069444444445},
 'frame21410': {'Lat': 39.54383333333333, 'Lon': 103.99863888888889},
 'frame13131': {'Lat': 38.94269444444444, 'Lon': 91.87630555555555},
 'frame12203': {'Lat': 38.67222222222222, 'Lon': 90.66386111111112},
 'frame2400': {'Lat': 37.801111111111105, 'Lon': 79.41436111111112},
 'frame26805': {'Lat': 40.41894444444444, 'Lon': 109.58152777777778},
 'frame24656': {'Lat': 39.55038888888889, 'Lon': 107.78122222222223},
 'frame34179': {'Lat': 44.51555555555556, 'Lon': 117.37480555555555},
 'frame37262': {'Lat': 46.41525, 'Lon': 120.30847222222222},
 'frame3809': {'Lat': 37.76766666666666, 'Lon': 81.03852777777777}}

In [429]:
json_txt = json.dumps(image_dict_final)
f = open("image_txt.json","w")
f.write(json_txt)
f.close()

In [ ]:
#print a 10-sample of key-value pairs from replaced_lons for keys that are also found in replaced_lats dictionary
{k: v for i, (k,v) in enumerate(dict((k, replaced_lons[k]) for k in (image_dict.keys() & bad_lats.keys())).items()) if i < 10}

In [193]:
#from https://stackoverflow.com/a/7205107/9987623
#function to perform nested merge of dictionaries
def merge_dicts(a, b, path=None):
    "merges b into a"
    if path is None: path = []
    for key in b:
        if key in a:
            if isinstance(a[key], dict) and isinstance(b[key], dict):
                merge(a[key], b[key], path + [str(key)])
            elif a[key] == b[key]:
                pass # same leaf value
            else:
                raise Exception('Conflict at %s' % '.'.join(path + [str(key)]))
        else:
            a[key] = b[key]
    return a

In [202]:
dict({k_lat: v_lat.update({'Lon': None}) for (k_lat,v_lat) in replaced_lats.items() if k_lat in (replaced_lats.keys() - 
    replaced_lons.keys())}.items() + {k_lon: v_lon.update({'Lat': None}) for (k_lon,v_lon) in 
    replaced_lons.items() if k_lon in (replaced_lons.keys() - replaced_lats.keys())}.items() + \
    {k_lat_lon: {'Lat': replaced_lats[k_lat_lon]['Lat'], 'Lon': replaced_lons[k_lat_lon]['Lon']} for k_lat_lon in 
     (replaced_lats.keys() & replaced_lons.keys())}.items())

TypeError: unsupported operand type(s) for +: 'dict_items' and 'dict_items'

In [228]:
#print a 10-sample of key-value pairs from replaced_lats
{k: v for i, (k,v) in enumerate(dict((k, replaced_lats[k]) for k in replaced_lats.keys()).items()) if i < 10}

{'frame30190': {'Lat': 41.84527777777778, 'Lon': None},
 'frame6817': {'Lat': 38.06769444444445, 'Lon': None},
 'frame10532': {'Lat': 38.316944444444445, 'Lon': None},
 'frame25589': {'Lat': 40.17252777777777, 'Lon': None},
 'frame13985': {'Lat': 38.95861111111111, 'Lon': None},
 'frame30184': {'Lat': 41.8385, 'Lon': None},
 'frame3953': {'Lat': 37.73105555555556, 'Lon': None},
 'frame13013': {'Lat': 38.91152777777778},
 'frame24867': {'Lat': 39.68708333333333, 'Lon': None},
 'frame17261': {'Lat': 38.878305555555556}}

In [210]:
{k_lat: v_lat.update({'Lon': None}) for (k_lat,v_lat) in replaced_lats.items() if k_lat in (replaced_lats.keys() - 
    replaced_lons.keys())}
#this probably modified the original replaced_lats dictionary

1496

In [342]:
{k_lat: v_lat.update({'Lon': None}) for (k_lat,v_lat) in replaced_lats.items()}
lats_not_lons = {k_lat: v_lat for (k_lat,v_lat) in replaced_lats.items() if k_lat 
                 in (replaced_lats.keys() - replaced_lons.keys())}
{k_lon: v_lon.update({'Lat': None}) for (k_lon,v_lon) in replaced_lons.items()}
lons_not_lats = {k_lon: v_lon for (k_lon,v_lon) in replaced_lons.items() if k_lon 
                 in (replaced_lons.keys() - replaced_lats.keys())}
lats_and_lons = {k_lat_lon: {'Lat': replaced_lats[k_lat_lon]['Lat'], 'Lon': replaced_lons[k_lat_lon]['Lon']} 
                 for k_lat_lon in (replaced_lats.keys() & replaced_lons.keys())}
dicts_to_merge = [lats_not_lons, lons_not_lats, lats_and_lons]
merged_replacements = dict(ChainMap(*dicts_to_merge))

# merged_replacements = {**{k_lat: v_lat.update({'Lon': None}) for (k_lat,v_lat) in replaced_lats.items() if k_lat in (replaced_lats.keys() - 
#     replaced_lons.keys())}, **{k_lon: v_lon.update({'Lat': None}) for (k_lon,v_lon) in 
#     replaced_lons.items() if k_lon in (replaced_lons.keys() - replaced_lats.keys())},
#     **{k_lat_lon: {'Lat': replaced_lats[k_lat_lon]['Lat'], 'Lon': replaced_lons[k_lat_lon]['Lon']} for k_lat_lon in 
#      (replaced_lats.keys() & replaced_lons.keys())}}

In [270]:
len(lats_and_lons)

256

In [271]:
len(lats_not_lons)

1496

In [345]:
len(replaced_lats.keys() - replaced_lons.keys())

1496

In [272]:
#print a 10-sample of key-value pairs from lats_not_lons
{k: v for i, (k,v) in enumerate(dict((k, lats_not_lons[k]) for k in lats_not_lons.keys()).items()) if i < 10}

{'frame30190': {'Lat': 41.84527777777778, 'Lon': None},
 'frame6817': {'Lat': 38.06769444444445, 'Lon': None},
 'frame10532': {'Lat': 38.316944444444445, 'Lon': None},
 'frame25589': {'Lat': 40.17252777777777, 'Lon': None},
 'frame13985': {'Lat': 38.95861111111111, 'Lon': None},
 'frame30184': {'Lat': 41.8385, 'Lon': None},
 'frame3953': {'Lat': 37.73105555555556, 'Lon': None},
 'frame24867': {'Lat': 39.68708333333333, 'Lon': None},
 'frame11176': {'Lat': 38.55972222222222, 'Lon': None},
 'frame16619': {'Lat': 38.992111111111114, 'Lon': None}}

In [343]:
merged_replacements

{'frame29245': {'Lon': 111.89122222222223, 'Lat': None},
 'frame2400': {'Lon': 79.41436111111112, 'Lat': None},
 'frame467': {'Lat': 38.819972222222226, 'Lon': None},
 'frame24656': {'Lat': 39.55038888888889, 'Lon': None},
 'frame5612': {'Lat': 38.40861111111111, 'Lon': None},
 'frame17386': {'Lon': 98.10030555555555, 'Lat': None},
 'frame9259': {'Lat': 38.200361111111114, 'Lon': None},
 'frame30075': {'Lat': 41.778666666666666, 'Lon': None},
 'frame30314': {'Lat': 41.91766666666666, 'Lon': None},
 'frame3809': {'Lon': 81.03852777777777, 'Lat': None},
 'frame1752': {'Lon': 78.88369444444444, 'Lat': None},
 'frame21531': {'Lon': 104.10308333333333, 'Lat': None},
 'frame35566': {'Lon': 118.60983333333333, 'Lat': None},
 'frame4182': {'Lon': 81.25583333333333, 'Lat': None},
 'frame21054': {'Lon': 103.66711111111111, 'Lat': None},
 'frame15056': {'Lon': 94.66708333333334, 'Lat': None},
 'frame37650': {'Lat': 46.72830555555556, 'Lon': None},
 'frame27286': {'Lat': 40.23255555555556, 'Lon': 

In [344]:
len(merged_replacements)

5841

In [277]:
sum(x is None for x in merged_replacements.values())

0

In [ ]:
{merge_dicts(copy.deepcopy(replaced_lats),copy.deepcopy(replaced_lons)) for replaced_lats}

merge_dicts(__, __) for dict(replaced_lats.items()

In [ ]:
{k_lat_lon: {'Lat': replaced_lats[k_lat_lon]['Lat'], 'Lon': replaced_lons[k_lat_lon]['Lon']} for k_lat_lon in replaced_lats.key() }

In [ ]:
'''
{'frame8359': {'Lat': None, 'Lon': 85.89155555555556},
 'frame6845': {'Lat': None, 'Lon': 84.04952777777778},
 'frame7862': {'Lat': None, 'Lon': 85.25738888888888},
 'frame32668': {'Lat': None, 'Lon': 115.90894444444444},
 'frame467': {'Lat': None, 'Lon': 77.46622222222223},
 'frame36963': {'Lat': None, 'Lon': None},
 'frame24656': {'Lat': None, 'Lon': 107.78122222222223},
 'frame6392': {'Lat': None, 'Lon': 83.43738888888889},
 'frame9776': {'Lat': None, 'Lon': 87.869},
 'frame5612': {'Lat': None, 'Lon': 82.36247222222222}}
'''

In [206]:
a_test = {'a':'foo','b':'bar','c':'baz'}
b_test = {'a':'spam','c':'ham','x':'blah'}

r_test = dict(a_test.items() + b_test.items())

TypeError: unsupported operand type(s) for +: 'dict_items' and 'dict_items'